In [ ]:
from tqdm import tqdm
import torchvision
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip '/content/drive/MyDrive/ML/data/images-test.zip'


Streaming output truncated to the last 5000 lines.
  inflating: images-test/Travel/0804143471.jpg  
  inflating: images-test/Travel/1558688374.jpg  
  inflating: images-test/Travel/1780096410.jpg  
  inflating: images-test/Travel/9643697371.jpg  
  inflating: images-test/Travel/1599473925.jpg  
  inflating: images-test/Travel/1490578951.jpg  
  inflating: images-test/Travel/8880294245.jpg  
  inflating: images-test/Travel/B008SMMFS2.jpg  
  inflating: images-test/Travel/0781810620.jpg  
  inflating: images-test/Travel/1771641460.jpg  
  inflating: images-test/Travel/1400018668.jpg  
  inflating: images-test/Travel/8408060287.jpg  
  inflating: images-test/Travel/1583550135.jpg  
  inflating: images-test/Travel/2700306872.jpg  
  inflating: images-test/Travel/0792268725.jpg  
  inflating: images-test/Travel/1559920300.jpg  
  inflating: images-test/Travel/8408062190.jpg  
  inflating: images-test/Travel/1612389724.jpg  
  inflating: images-test/Travel/B006JATBYQ.jpg  
  inflating: image

In [ ]:
!unzip '/content/drive/MyDrive/ML/data/images-train.zip'


Streaming output truncated to the last 5000 lines.
  inflating: images-train/Science & Math/0822341077.jpg  
  inflating: images-train/Science & Math/0395906636.jpg  
  inflating: images-train/Science & Math/0412989417.jpg  
  inflating: images-train/Science & Math/393976728X.jpg  
  inflating: images-train/Science & Math/0934718660.jpg  
  inflating: images-train/Science & Math/111183430X.jpg  
  inflating: images-train/Science & Math/0871699532.jpg  
  inflating: images-train/Science & Math/0201054337.jpg  
  inflating: images-train/Science & Math/0939251981.jpg  
  inflating: images-train/Science & Math/0849325234.jpg  
  inflating: images-train/Science & Math/0881920002.jpg  
  inflating: images-train/Science & Math/0321811127.jpg  
  inflating: images-train/Science & Math/038750298X.jpg  
  inflating: images-train/Science & Math/012044982X.jpg  
  inflating: images-train/Science & Math/1583242759.jpg  
  inflating: images-train/Science & Math/0133840549.jpg  
  inflating: images-t

In [ ]:
!rm "images-test/Children's Books/0195215818.jpg"
!rm "images-test/Comics & Graphic Novels/1421534673.jpg"
!rm "images-test/Sports & Outdoors/1847329802.jpg"

In [ ]:
rm -rf images-train/Category

In [ ]:
import os
import os.path as osp

IMAGE_VAL_BASE_PATH = 'images-val'
IMAGE_TRAIN_BASE_PATH = 'images-train'

if not osp.exists(IMAGE_VAL_BASE_PATH):
    os.mkdir(IMAGE_VAL_BASE_PATH)

for cat in os.listdir(IMAGE_TRAIN_BASE_PATH):
    image_val_cat_path = osp.join(IMAGE_VAL_BASE_PATH, cat)
    if not osp.exists(image_val_cat_path):
        os.mkdir(image_val_cat_path)
    image_cat_path = osp.join(IMAGE_TRAIN_BASE_PATH, cat)
    imgs = os.listdir(image_cat_path)
    num_img = len(imgs)
    take = int((1/9) * num_img)
    for i in range(take):
        img_name = imgs[i]
        img_path = osp.join(image_cat_path, img_name)
        os.rename(img_path, osp.join(image_val_cat_path, img_name))

In [ ]:
import torch
import torchvision

In [ ]:
len(train_dataloader)

NameError: ignored

In [ ]:
# import torchvision
# import torch

# train_dataset = torchvision.datasets.ImageFolder(
#     'images-train',
#     transform=torchvision.transforms.Compose([
#         # Resize step is required as we will use a ResNet model, which accepts at leats 224x224 images
#         torchvision.transforms.Resize((224,224)),
#         torchvision.transforms.ToTensor(),
#     ]),
#     # is_valid_file=lambda x: True if any(fname.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.ppm', '.pgm', '.tif', '.tiff', '.webp')) for fname in os.listdir(x)) else False

# )

# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=False, num_workers=2, pin_memory=True)

# means = []
# stdevs = []
# for X, _ in tqdm(train_dataloader):
#     # Dimensions 0,2,3 are respectively the batch, height and width dimensions
#     means.append(X.mean(dim=(0,2,3)))
#     stdevs.append(X.std(dim=(0,2,3)))

# mean = torch.stack(means, dim=0).mean(dim=0)
# stdev = torch.stack(stdevs, dim=0).mean(dim=0)

100%|██████████| 122/122 [05:11<00:00,  2.55s/it]


In [ ]:
mean, stdev = torch.Tensor([0.5483, 0.5152, 0.4714]), torch.Tensor([0.3267, 0.3070, 0.3118])

In [ ]:
train_transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224)),
        torchvision.transforms.AutoAugment(policy=torchvision.transforms.AutoAugmentPolicy.CIFAR10),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean, stdev)
    ])
test_transforms = torchvision.transforms.Compose([
        torchvision.transforms.Resize((224,224)),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean, stdev)
    ])
train_dataset, val_dataset = [torchvision.datasets.ImageFolder(folder, transform=train_transforms) for folder in ['images-train', 'images-val']]
test_dataset = torchvision.datasets.ImageFolder('images-test', transform=test_transforms)

In [ ]:
num_gpus = torch.cuda.device_count()

train_dataloader = (torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2*num_gpus, pin_memory=True))
val_dataloader = (torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=2*num_gpus, pin_memory=True))
test_dataloader = (torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2*num_gpus, pin_memory=True))

In [ ]:
def get_net():
    model_ft = torchvision.models.resnet50(pretrained=True)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = torch.nn.Linear(num_ftrs, 10)
    return model_ft.to('cuda')

In [ ]:
import time
from tqdm import tqdm
import shutil




def train(net, train_dataloader, val_dataloader, criterion, optimizer,
          scheduler=None, scaler=None, epochs=10, patient=5, device='gpu',
          checkpoint_epochs=50, start_epoch=1, max_val_acc=0):
    start = time.time()
    print(f'Training for {epochs} epochs on {device}')
    current = 0
    best_val_epoch = start_epoch
    use_amp = True

    # dataloader_progressbar = tqdm(train_dataloader)
    # val_dataloader_progressbar = tqdm(val_dataloader)

    for epoch in range(start_epoch, epochs+1):
        print(f"Epoch {epoch}/{epochs}")

        net.train()  # put network in train mode for Dropout and Batch Normalization
        train_loss = torch.tensor(0., device=device)  # loss and accuracy tensors are on the GPU to avoid data transfers
        train_accuracy = torch.tensor(0., device=device)
        val_loss = torch.tensor(0., device=device)
        val_accuracy = torch.tensor(0., device=device)
        for X, y in tqdm(train_dataloader):
            X = X.to(device)
            y = y.to(device)
            with torch.autocast(device_type=device, dtype=torch.float16, enabled=use_amp):
                preds = net(X)
                loss = criterion(preds, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            # optimizer.zero_grad()
            # loss.backward()
            # optimizer.step()
            optimizer.zero_grad(set_to_none=True)

            with torch.no_grad():
                train_loss += loss * train_dataloader.batch_size
                train_accuracy += (torch.argmax(preds, dim=1) == y).sum()
        with torch.no_grad():
            for X, y in tqdm(val_dataloader):
                X = X.to(device)
                y = y.to(device)
                preds = net(X)
                loss = criterion(preds, y)
                val_loss += loss * val_dataloader.batch_size
                val_accuracy += (torch.argmax(preds, dim=1)==y).sum()


        if scheduler is not None:
            scheduler.step()

        print(f'Training loss: {train_loss/len(train_dataloader.dataset):.2f}')
        print(f'Training accuracy: {100*train_accuracy/len(train_dataloader.dataset):.2f}')

        epoch_val_acc = 100*val_accuracy/len(val_dataloader.dataset)
        print(f'Val accuracy: {epoch_val_acc:.2f}')
        print(f'Best val accuracy: {max_val_acc}')

        if epoch_val_acc <= max_val_acc:
            current+=1
            if current==patient:
                print(f"reload checkpoint {best_val_epoch} with val accuracy {max_val_acc}")
                checkpoint = torch.load(f'/content/drive/MyDrive/ML/ckpt/khoa/resnet50/checkpoint_best.pth.tar')
                net.load_state_dict(checkpoint['state_dict'])
                optimizer.load_state_dict(checkpoint['optimizer'])
                scaler.load_state_dict(checkpoint['scaler'])
                current=0
        else:
            current = 0
            max_val_acc = epoch_val_acc
            best_val_epoch = epoch
            print(f"Best epoch {epoch} with val accuracy {epoch_val_acc}")
            torch.save({
                'epoch': epoch,
                'state_dict': net.state_dict(),
                'optimizer': optimizer.state_dict(),
                "scaler": scaler.state_dict(),
                "val_acc": max_val_acc
            }, f'/content/drive/MyDrive/ML/ckpt/khoa/resnet50/checkpoint_best.pth.tar')

        # if epoch%checkpoint_epochs==0:
        #     torch.save({
        #         'epoch': epoch,
        #         'state_dict': net.state_dict(),
        #         'optimizer': optimizer.state_dict(),
        #     }, f'/content/drive/MyDrive/ML/ckpt/tho/vgg16_12k/checkpoint_{epoch}.pth.tar')

        print()

    end = time.time()
    print(f'Total training time: {end-start:.1f} seconds')
    return net

In [ ]:
checkpoint = torch.load(f'/content/drive/MyDrive/ML/ckpt/khoa/vgg16_12k/checkpoint_best.pth.tar')

In [ ]:
checkpoint["epoch"]

11

In [ ]:
def resume_train():
    model = get_net().to(device)
    criterion = torch.nn.CrossEntropyLoss()
    use_amp = True
    scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
    optimizer = torch.optim.Adam(model.parameters())
    # checkpoint = torch.load(f'/content/drive/MyDrive/ML/ckpt/khoa/vgg16_12k/checkpoint_best.pth.tar')
    # model.load_state_dict(checkpoint['state_dict'])
    # optimizer.load_state_dict(checkpoint['optimizer'])
    # scaler.load_state_dict(checkpoint['scaler'])
    # epoch = checkpoint["epoch"]
    # max_val_acc = checkpoint["val_acc"]
    epoch = 1
    max_val_acc=0
    return model, criterion, optimizer, scaler, epoch, max_val_acc

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'gpu'
lr, weight_decay, epochs = 1e-5, 5e-4, 50


In [ ]:
model, criterion, optimizer, scaler, epoch, max_val_acc = resume_train()
model = model.to(device)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:

net = train(model, train_dataloader=train_dataloader, val_dataloader=val_dataloader,
            criterion=criterion, optimizer=optimizer, scaler=scaler, scheduler=None,
            epochs=epochs, patient=10, device=device, checkpoint_epochs=1, start_epoch=epoch,
            max_val_acc=max_val_acc)

Training for 50 epochs on cuda
Epoch 1/50


100%|██████████| 61/61 [00:47<00:00,  1.28it/s]


Training loss: 1.51
Training accuracy: 48.70
Val accuracy: 20.53
Best val accuracy: 0
Best epoch 1 with val accuracy 20.52861213684082

Epoch 2/50


100%|██████████| 61/61 [00:47<00:00,  1.27it/s]


Training loss: 1.31
Training accuracy: 55.66
Val accuracy: 19.39
Best val accuracy: 20.52861213684082

Epoch 3/50


...

100%|██████████| 61/61 [00:43<00:00,  1.42it/s]


Training loss: 0.28
Training accuracy: 90.68
Val accuracy: 23.81
Best val accuracy: 24.056968688964844

Epoch 29/50


100%|██████████| 61/61 [00:40<00:00,  1.51it/s]


Training loss: 0.27
Training accuracy: 91.02
Val accuracy: 23.76
Best val accuracy: 24.056968688964844
reload checkpoint 19 with val accuracy 24.056968688964844

Epoch 30/50


 50%|█████     | 245/488 [03:10<02:44,  1.48it/s]

In [ ]:
# model = get_net().to(device)

# Standard CrossEntropy Loss for multi-class classification problems
criterion = torch.nn.CrossEntropyLoss()
use_amp = True
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)
optimizer = torch.optim.Adam(model.parameters())

net = train(model, train_dataloader=train_dataloader, val_dataloader=val_dataloader,
            criterion=criterion, optimizer=optimizer, scaler=scaler, scheduler=None, epochs=epochs,
            patient=3, device=device)


In [ ]:
!ls /content/drive/MyDrive/ML/ckpt/khoa/vgg16_12k/

ls: cannot access '/content/drive/MyDrive/ML/ckpt/khoa/vgg16_12k/': No such file or directory


In [ ]:
import torch
torch.save({
  'epoch': 44,
  'state_dict': model.state_dict(),
  'optimizer': optimizer.state_dict(),
}, f'/content/drive/MyDrive/ML/ckpt/khoa/vgg16_12k/checkpoint_best.pth.tar')


In [ ]:
# model.load_state_dict(torch.load('/content/drive/MyDrive/ML/ckpt/tho/vgg16_test/checkpoint_10.pth.tar')['state_dict'])


In [ ]:
preds = []
y_lst = []
i=0
model.eval()
with torch.no_grad():
    for X, y in test_dataloader:
        i+=1
        X = X.to(device)
        preds.extend(model(X).argmax(dim=1).type(torch.int32).cpu().numpy())
        y_lst.extend(y.numpy())

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(preds, y_lst)
accuracy

0.590542800668998